In [6]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
import pandas as pd
import sys

from CASutils import filter_utils as filt
from CASutils import readdata_utils as read
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [7]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '10GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=10GB',
    project='P04010022',
    walltime='03:00:00',
    interface='ext')

# scale up
cluster.scale(12)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44069 instead
  f"Port {expected} is already in use.\n"


In [13]:
cluster

PBSCluster(c2b695af, 'tcp://128.117.208.96:34959', workers=12, threads=12, memory=111.72 GiB)

In [14]:
client

<Client: 'tcp://128.117.208.96:34959' processes=12 threads=12, memory=111.72 GiB>

### Read in the data

In [15]:
ystart=1979 ; yend=2019

pathout="/glade/campaign/cgd/cas/islas/python/L83/DATA_SORT/MJO_OMEGA500/"

In [ ]:
for imem in np.arange(1,3+1,1):
    basepathhist="/glade/campaign/cesm/development/cvcwg/cvwg/L83/timeseries/f.e21.FHIST_BGC.f09_f09_mg17.L83_cam6."+str(imem).zfill(3)+"/atm/proc/tseries/day_1/"
    uzmhist = xr.open_mfdataset(basepathhist+"*.Uzm.*.nc")
    uzmhist = read.fixcesmtime(uzmhist)
    omegahist = xr.open_mfdataset(basepathhist+"*.OMEGA500.*.nc")
    omegahist = read.fixcesmtime(omegahist)
    omegahist['lon'] = uzmhist.lon ; omegahist['lat'] = uzmhist.lat

    basepathssp = "/glade/campaign/cesm/development/cvcwg/cvwg/L83/timeseries/f.e21.FHIST_BGC.f09_f09_mg17.L83_cam6_SSP370."+str(imem).zfill(3)+"/atm/proc/tseries/day_1/"
    uzmssp = xr.open_mfdataset(basepathssp+"*.Uzm.*.nc")
    uzmssp = read.fixcesmtime(uzmssp)
    omegassp = xr.open_mfdataset(basepathssp+"*.OMEGA500.*.nc")
    omegassp = read.fixcesmtime(omegassp)
    omegassp['lon'] = uzmhist.lon ; omegassp['lat'] = uzmhist.lat

    uzm = xr.concat([uzmhist, uzmssp], dim='time')
    omega = xr.concat([omegahist, omegassp], dim='time')

    uzm = uzm.sel(time=slice(str(ystart)+"-01-01",str(yend+1)+"-12-31"))
    uzm = uzm.Uzm.isel(zlon=0)

    omega = omega.sel(time=slice(str(ystart)+"-01-01",str(yend+1)+"-12-31"))
    omega = omega.OMEGA500

    alluzm=[]
    allomega=[]
    for iyear in np.arange(ystart,yend+1,1):
        print(iyear)
        uzmuse = uzm.sel(time=slice(str(iyear)+"-12-01",str(iyear+1)+"-02-28")).mean('time')
        alluzm.append(uzmuse)

        omegause = omega.sel(time=slice(str(iyear)+"-11-01",str(iyear+1)+"-03-31"))
        omegaclim = omegause.mean('time')
        omegaanoms = omegause - omegaclim
        omegaanoms['time'] = pd.date_range("1970-11-01","1971-03-31")
        mjoomega = filt.wkfilter(omegaanoms, 0.15, 1, 5, 20, 100, spd=1)
        mjoomega = mjoomega.sel(time=slice("1970-12-01","1971-02-28"))
        allomega.append(mjoomega)

    year = xr.DataArray(np.arange(ystart+1,yend+2,1), coords=[np.arange(ystart+1, yend+2, 1)], dims=['year'], name='year')
    allomega = xr.concat(allomega, dim=year)
    alluzm = xr.concat(alluzm, dim=year)
    allomega = allomega.rename('MJO_OMEGA500')
    alluzm = alluzm.rename('Uzm')

    ds = xr.merge([allomega, alluzm])
    ds.to_netcdf(pathout+'MJO_OMEGA500_FL83_'+str(ystart)+'_'+str(yend+1)+'_'+str(imem).zfill(3)+'.nc')

cluster.close()

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994


In [18]:
print(basepathssp+"*.Uzm.*.nc")

/glade/campaign/cesm/development/cvcwg/L83/timeseries/f.e21.FHIST_BGC.f09_f09_mg17.L83_cam6_SSP370.001/atm/proc/tseries/day_1/*.Uzm.*.nc


In [17]:
print(basepathssp)

/glade/campaign/cesm/development/cvcwg/L83/timeseries/f.e21.FHIST_BGC.f09_f09_mg17.L83_cam6_SSP370.001/atm/proc/tseries/day_1/
